In [12]:
import dir_constants as dc
from tqdm import tqdm_notebook
import time

In [2]:
platform = 'lendingclub'

store = pd.HDFStore('/home/justin/lendingclub.h5',append=True)
pmt_hist = store['pmt_hist_ready']
store.close()

In [4]:
# some setups for code block below
pmt_hist_ids = pmt_hist['loan_id'].astype(int)
max_id = pmt_hist_ids.max()
chunksize = 800
n_chunks = len(pmt_hist_ids)//chunksize + 1

interesting_cols_over_time = [
    'outs_princp_beg',
    'all_cash_to_inv',
    'date',
    'fico_last',
    'm_on_books',
    'status_period_end',
    'loan_id',
]

disc_rates = [.07]#np.arange(.05,.36,.01)

In [5]:
pmt_hist = pmt_hist[interesting_cols_over_time]

In [22]:
id_grouped = pmt_hist.groupby('loan_id')
start_time = time.time()
npv_roi_holder = {}
k = 0
for ids, group in id_grouped:
    npv_roi_dict = {}
    funded = group.iat[0,0]
    cfs = [-funded] + group['all_cash_to_inv'].tolist()
    for rate in disc_rates:
        npv_roi_dict[rate] = np.npv(rate/12, cfs)/funded
    npv_roi_holder[ids] = npv_roi_dict
    if k % 10000 == 1:
        elapsed_time = time.time() - start_time
        print(elapsed_time)
    k += 1

2.9654183387756348
6.845083475112915
10.59197449684143
14.356499195098877
18.50809907913208
22.300944328308105
26.18256950378418
30.081371068954468
33.90814161300659
37.76029324531555
41.733572483062744
45.57996368408203
49.35684394836426
53.345433950424194
57.22968530654907
61.170145750045776
65.02333569526672
68.85399961471558
72.69620275497437
76.64130711555481
80.58393478393555
84.44846153259277
88.3788571357727
92.18458414077759
95.96855545043945
99.83124351501465
103.81852769851685
107.69868755340576
111.68504166603088
115.50619983673096
119.37564206123352
123.22204947471619
127.01842188835144
130.99366068840027
134.94616985321045
138.81997060775757
142.79558753967285
146.59764695167542
150.49200201034546
154.32513785362244
158.36717915534973
162.29710817337036
166.1970031261444
170.0865762233734
173.930504322052
177.97809505462646
181.9148395061493
185.7598729133606
189.59403467178345
193.36831402778625
197.18681144714355
201.05986547470093
205.00990843772888
208.82778692245483


In [23]:
npv_roi_df = pd.DataFrame(npv_roi_holder).T
npv_roi_df.columns = npv_roi_df.columns.values.round(2)

In [24]:
npv_roi_df.shape

(1646778, 1)

In [27]:
store.open()
store['loan_npv_rois'] = npv_roi_df
store.close()